In [ ]:
import pickle as pkl
from QAOAUtils import *
from tqdm import tqdm
from tqdm.contrib import itertools
from tqdm.contrib.itertools import product
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
from matplotlib.gridspec import GridSpec
import pandas as pd

In [ ]:
Random_QUBO_DATA= pkl.load( open ("RandomQUBODepth0Test.p", "rb") )
TSP_MIS_DATA= pkl.load( open ("TSPMISDepth0Test.p", "rb") )
PO_DATA= pkl.load( open ("PortfolioOptDepth0Test.p", "rb") )

p_data = {'Continous':Random_QUBO_DATA[0:4],'Discrete':Random_QUBO_DATA[4:8]} | {'TSP':TSP_MIS_DATA[0:4]} | {'Portfolio Optimization':PO_DATA[0:4]} | {'MIS-GNP':TSP_MIS_DATA[4:8],'MIS-NWS':TSP_MIS_DATA[8:12]}
A_list = {prob:p_data[prob][-1] for prob in p_data.keys()}
M_list= {prob:[p_data[prob][2][i][-1] for i in range(1000)] for prob in p_data.keys()}

m_list = {k:[] for k in p_data.keys()}
for prob,idx in product(p_data.keys(),range(1000)):
        A = A_list[prob][idx]
        m_list[prob].append(-brute_force_maxcut(-A)[-1] )

In [ ]:
M_list= {prob:[p_data[prob][2][i][-1] for i in range(1000)] for prob in p_data.keys()}
res = 101
counts = {prob:[0]*res for prob in p_data.keys()}
for prob,idx in product(p_data.keys(),range(1000)):
    A = A_list[prob][idx]
    M = M_list[prob][idx]
    m = m_list[prob][idx]
    pre = pre_compute(A)
    for i in np.floor(((pre-m)/(M-m))*res):
        counts[prob][min(int(i),res-1)]+=1/(2**17 * 1000)
    

In [ ]:
fig,ax = plt.subplots(1,len(p_data.keys()),figsize=(20,3),sharey=False)
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
for (i,prob) in enumerate(p_data.keys()):
    ax[i].stairs(np.array(counts[prob])/np.max(counts[prob]),np.linspace(0,1,res+1),fill=True)
    ax[i].set_xlabel(r'${\alpha}$', fontsize=13)
    ax[i].set_title(prob, fontsize=14)
    if(i>0):
        ax[i].set_yticks([])
    else:
        ax[i].set_ylabel(r'$P/P_\mathrm{max}$', fontsize=13)
    ax[i].text(0.5,-0.3, r'$P_\mathrm{max}=$' + str(np.round(np.max(counts[prob])*res,2)), size=14, ha="center", 
         transform=ax[i].transAxes)
plt.subplots_adjust(wspace=0) 
plt.savefig(r'Figures\AlphaDistributions.pdf',dpi=300,bbox_inches="tight")
plt.show()